## Paquetes y datos originales ## 

In [56]:
import json
import pandas as pd
from datetime import datetime
from listas import disfagia_list, PA_list, P_list
from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.proportion import proportions_ztest

### Previamente he tratado la base de datos, creando 16 nuevas columnas para facilitar la lectura de los datos

In [57]:
tabla.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,10.0,10.0,None,NaN,NaN,NaN,NaN,NaN,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,1.0,1.0,None,58.308333,NaN,0.0,1.0,0.0,Líquid,Alt
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,6.0,6.0,None,NaN,NaN,NaN,NaN,NaN,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,8.0,8.0,None,NaN,NaN,1.0,1.0,1.0,NECTAR,ALT
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,7.0,7.0,4.0,49.900000,NaN,1.0,NaN,NaN,None,None


## Dividiré los datos en 3 grupos:
- Con PA (codigo ICD de PA )
- Con PA + MECVV (MECVV positivo + icd pneumonia < 30 days)
- Sin PA (ICD pneumonia or icd pneumonia + mecvv positive > 30 days)

# CON PA 

In [58]:
tabla.PA_diagnosticada.value_counts()

PA_diagnosticada
0.0    33
1.0    17
Name: count, dtype: int64

In [59]:
CON_PA = tabla[tabla["PA_diagnosticada"] == 1.0]
CON_PA.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,10.0,10.0,None,NaN,NaN,NaN,NaN,NaN,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,1.0,1.0,None,58.308333,NaN,0.0,1.0,0.0,Líquid,Alt
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,8.0,8.0,None,NaN,NaN,1.0,1.0,1.0,NECTAR,ALT
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,9.0,9.0,None,NaN,NaN,1.0,1.0,0.0,LIQUID,BAIX
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,7.0,7.0,None,46.400000,NaN,1.0,1.0,1.0,NECTAR,MIG
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,6.0,6.0,None,NaN,7.0,1.0,1.0,1.0,PUDING,MIG
15,75,M,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-09-19 00...",[],...,13.0,13.0,None,NaN,NaN,NaN,NaN,NaN,None,None
26,97,F,0,"[{'date': '20220419', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'C09BA', 'dataInici': '2022-04-17 00...",[],...,5.0,5.0,None,NaN,NaN,NaN,NaN,NaN,None,None
28,95,F,0,"[{'date': '20220526', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2022-05-30'}]","[{'codi': 'C10AA', 'dataInici': '2020-11-18 00...","[{'data': '20220529115841', 'disfagia': 'S', '...",...,6.0,6.0,None,0.000000,NaN,0.0,0.0,0.0,LIQUID,MIG
29,97,F,0,[],[],"[{'activitat': '3', 'estatMental': '2', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2020-11-30 00...",[],...,14.0,14.0,None,NaN,NaN,NaN,NaN,NaN,None,None


# CON_PA + MECVV

In [60]:
CON_PA_MECVV = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30)]
CON_PA_MECVV.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,8.0,8.0,None,NaN,NaN,1.0,1.0,1.0,NECTAR,ALT
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,9.0,9.0,None,NaN,NaN,1.0,1.0,0.0,LIQUID,BAIX
12,86,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '46.4', 'data': '2020-09-09'}]","[{'codi': 'G04CA', 'dataInici': '2020-09-04 00...","[{'data': '20200907110400', 'disfagia': 'S', '...",...,7.0,7.0,None,46.40,NaN,1.0,1.0,1.0,NECTAR,MIG
13,71,F,0,"[{'date': '20220216', 'name': 'CULTIU Esput', ...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...","[{'dataValoracio': '2020-09-06', 'horaValoraci...",[],"[{'codi': 'C10AA', 'dataInici': '2020-09-06 00...","[{'data': '20200906175506', 'disfagia': 'N', '...",...,6.0,6.0,None,NaN,7.0,1.0,1.0,1.0,PUDING,MIG
27,100,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],"[{'valor': '0', 'data': '2021-09-14'}, {'valor...","[{'codi': 'C08CA', 'dataInici': '2020-11-23 00...","[{'data': '20201125131827', 'disfagia': 'S', '...",...,9.0,9.0,None,20.55,NaN,1.0,1.0,0.0,LIQUID,BAIX
37,86,M,0,"[{'date': '20221003', 'name': 'GLUCOSA Sèrum',...","[{'imc': '', 'anorexia': '', 'perduaPes': '', ...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '72', 'data': '2022-10-09'}]","[{'codi': 'G04CA', 'dataInici': '2022-09-26 00...","[{'data': '20221012122425', 'disfagia': 'S', '...",...,9.0,9.0,0.0,72.00,NaN,1.0,0.0,1.0,NECTAR,MIG
39,87,F,0,"[{'date': '20230619', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'N02BE', 'dataInici': '2021-01-17 00...","[{'data': '20220111085147', 'disfagia': 'S', '...",...,4.0,4.0,None,NaN,NaN,1.0,1.0,0.0,LIQUID,MIG
41,93,F,0,"[{'date': '20221013', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],"[{'valor': '0', 'data': '2023-06-02'}, {'valor...","[{'codi': 'C10AA', 'dataInici': '2021-02-05 00...","[{'data': '20230530155654', 'disfagia': 'S', '...",...,5.0,5.0,None,30.00,NaN,1.0,0.0,1.0,LIQUID,BAIX
46,86,M,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],"[{'valor': '55.1', 'data': '2021-03-21'}]","[{'codi': 'G04CA', 'dataInici': '2021-03-20 00...","[{'data': '20210721140136', 'disfagia': 'S', '...",...,10.0,10.0,None,55.10,NaN,1.0,1.0,1.0,PUDING,MIG
47,94,M,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-03-23 00...","[{'data': '20210325091551', 'disfagia': 'S', '...",...,9.0,9.0,None,NaN,NaN,1.0,0.0,1.0,LIQUID,BAIX


# SIN_PA

In [61]:
SIN_PA = tabla[(tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30) | (tabla["P_diagnosticada"] == 1.0)]
SIN_PA.head(len(tabla))

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,10.0,10.0,None,NaN,NaN,NaN,NaN,NaN,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,1.0,1.0,None,58.308333,NaN,0.0,1.0,0.0,Líquid,Alt
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,6.0,6.0,None,NaN,NaN,NaN,NaN,NaN,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,8.0,8.0,None,NaN,NaN,1.0,1.0,1.0,NECTAR,ALT
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,7.0,7.0,4.0,49.900000,NaN,1.0,NaN,NaN,None,None
5,57,M,0,"[{'date': '20220909', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '1', 'estatMental': '0', 'humit...",[],"[{'valor': '83.7', 'data': '2020-08-13'}, {'va...","[{'codi': 'N02BE', 'dataInici': '2020-08-12 00...",[],...,3.0,3.0,None,77.760000,NaN,NaN,NaN,NaN,None,None
6,101,F,0,[],[],"[{'activitat': '3', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C09AA', 'dataInici': '2021-08-21 00...","[{'data': '20210819124116', 'disfagia': 'S', '...",...,9.0,9.0,None,NaN,NaN,1.0,1.0,0.0,LIQUID,BAIX
7,90,F,0,"[{'date': '20220627', 'name': 'GLUCOSA Sèrum',...","[{'imc': '0', 'anorexia': '0', 'perduaPes': '1...","[{'activitat': '3', 'estatMental': '1', 'humit...","[{'dataValoracio': '2020-07-30', 'horaValoraci...",[],"[{'codi': 'C09AA', 'dataInici': '2020-07-31 00...","[{'data': '20200731092102', 'disfagia': 'N', '...",...,12.0,12.0,1.0,NaN,9.0,0.0,0.0,0.0,LIQUID,MIG
8,90,F,0,"[{'date': '20220827', 'name': 'GLUCOSA Sèrum',...",[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'N02BB', 'dataInici': '2023-07-17 00...",[],...,5.0,5.0,None,NaN,NaN,NaN,NaN,NaN,None,None
9,91,M,0,[],[],"[{'activitat': '1', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'C10AA', 'dataInici': '2020-08-25 00...",[],...,3.0,3.0,None,NaN,NaN,NaN,NaN,NaN,None,None


Sin aplicar la ultima condicion OR, es decir, queriendo nos devuelva solo las filas que cumplan las 3 condiciones, nos devuelve la: 4, 25, 42, 45 y 48. Sino, nos devuelve todas menos la 43, la cual no tiene un diagnostico de pneumonia.

Ahora que he distribuido los datos en 3 grupos, puedo empezar a analizar su contenido


## Número

In [62]:
num_CON_PA = len(CON_PA)
num_CON_PA_MECVV= len(CON_PA_MECVV)
num_SIN_PA = len(SIN_PA)

print(num_CON_PA, num_CON_PA_MECVV, num_SIN_PA)

17 10 49


Esto también se puede ver con tan solo mirar las filas que han salido en el apartado anterior, donde vemos que la tabla de CON_PA tiene 17 filas, la de CON_PA_MECVV tiene 10 y la de SIN_PA tiene 49.

#### Pvalor

In [63]:
z_stat, p_val = proportions_ztest(num_CON_PA, num_CON_PA_MECVV, value=0)
print("P-value para CON_PA vs CON_PA_MECVV:", p_val)
#TODO: es incorrecto?

P-value para CON_PA vs CON_PA_MECVV: nan


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\statsmodels\stats\proportion.py:1025: RuntimeWarning: invalid value encountered in sqrt
  std_diff = np.sqrt(var_)


In [64]:
z_stat, p_val = proportions_ztest(num_CON_PA, num_SIN_PA, value=0)
print("P-value para CON_PA vs SIN_PA:", p_val)

P-value para CON_PA vs SIN_PA: 3.359355153724907e-07


In [65]:
z_stat, p_val = proportions_ztest(num_CON_PA_MECVV, num_SIN_PA, value=0)
print("P-value para CON_PA_MECVV vs SIN_PA:", p_val)

P-value para CON_PA_MECVV vs SIN_PA: 0.0003932275362231457


## Edat

In [66]:
edad_CON_PA = CON_PA['edat'].mean()
desviacion_estandar_edad = CON_PA['edat'].std()

resultado = f"{edad_CON_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Promedio ± Desviación Estándar de la Edad:", resultado)


Promedio ± Desviación Estándar de la Edad: 87.06 ± 8.99


In [67]:
edad_CON_PA_MECVV = CON_PA_MECVV['edat'].mean()
desviacion_estandar_edad = CON_PA_MECVV['edat'].std()

resultado = f"{edad_CON_PA_MECVV:.2f} ± {desviacion_estandar_edad:.2f}"
print("Promedio ± Desviación Estándar de la Edad:", resultado)

Promedio ± Desviación Estándar de la Edad: 88.90 ± 8.67


In [68]:
edad_SIN_PA = SIN_PA['edat'].mean()
desviacion_estandar_edad = SIN_PA['edat'].std()

resultado = f"{edad_SIN_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Promedio ± Desviación Estándar de la Edad:", resultado)

Promedio ± Desviación Estándar de la Edad: 80.12 ± 17.01


#### Pvalor

In [69]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['edat'], CON_PA_MECVV['edat'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)


P-valor entre CON_PA y CON_PA_MECVV: 0.6051223082851969


In [70]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['edat'], SIN_PA['edat'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA", p_val_13)

P-valor entre CON_PA y SIN_PA 0.038275970403095716


In [71]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['edat'], SIN_PA['edat'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.024173860274705212


## Sexe

In [72]:
sexe_CON_PA = CON_PA['sexe'].value_counts()
sexe_CON_PA_MECVV = CON_PA_MECVV['sexe'].value_counts()
sexe_SIN_PA = SIN_PA['sexe'].value_counts()

In [73]:
if 'F' in sexe_CON_PA:
    percent_female1 = (sexe_CON_PA['F'] / len(CON_PA)) * 100
    print(f"Porcentaje de sexo femenino en CON_PA: {percent_female1:.2f}%")
else:
    print("No hay datos de sexo femenino en el CON_PA")

Porcentaje de sexo femenino en CON_PA: 47.06%


In [74]:
if 'F' in sexe_CON_PA_MECVV:
    percent_female2 = (sexe_CON_PA_MECVV['F'] / len(CON_PA)) * 100
    print(f"Porcentaje de sexo femenino en DataFrame 2: {percent_female2:.2f}%")
else:
    print("No hay datos de sexo femenino en el DataFrame 2.")

Porcentaje de sexo femenino en DataFrame 2: 23.53%


In [75]:
if 'F' in sexe_SIN_PA:
    percent_female3 = (sexe_SIN_PA['F'] / len(SIN_PA)) * 100
    print(f"Porcentaje de sexo femenino en DataFrame 3: {percent_female3:.2f}%")
else:
    print("No hay datos de sexo femenino en el DataFrame 3.")


Porcentaje de sexo femenino en DataFrame 3: 34.69%


#### Pvalor

In [76]:
persexe_CON_PA = 47.06
persexe_CON_PA_MECVV = 23.53
persexe_SIN_PA= 34.69

In [77]:
z_stat, p_val = proportions_ztest([47.06, 23.53], [100, 100])
print("P-value para CON_PA vs CON_PA_MECVV:", p_val)

P-value para CON_PA vs CON_PA_MECVV: 0.0004983897114131808


In [78]:
z_stat, p_val = proportions_ztest([47.06, 34.69], [100, 100])
print("P-value para CON_PA vs SIN_PA:", p_val)

P-value para CON_PA vs SIN_PA: 0.0751965905655568


In [79]:
z_stat, p_val = proportions_ztest([23.53, 34.69], [100, 100])
print("P-value para CON_PA_MECVV vs SIN_PA:", p_val)

P-value para CON_PA_MECVV vs SIN_PA: 0.08236190815486331


## Ingressos

In [80]:
promedio_ingresos = CON_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = CON_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Promedio ± Desviación Estándar del Número de Ingresos:", resultado)

Promedio ± Desviación Estándar del Número de Ingresos: 5.59 ± 3.37


In [81]:
promedio_ingresos = CON_PA_MECVV['Num_ingresos'].mean()
desviacion_estandar_ingresos = CON_PA_MECVV['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Promedio ± Desviación Estándar del Número de Ingresos:", resultado)

Promedio ± Desviación Estándar del Número de Ingresos: 5.90 ± 2.08


In [82]:
promedio_ingresos = SIN_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = SIN_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Promedio ± Desviación Estándar del Número de Ingresos:", resultado)

Promedio ± Desviación Estándar del Número de Ingresos: 6.69 ± 5.13


## Pvalor

In [83]:
#ANOVA 
ingresos_CON_PA = CON_PA['Num_ingresos']
ingresos_CON_PA_MECVV = CON_PA_MECVV['Num_ingresos']
ingresos_SIN_PA = SIN_PA['Num_ingresos']

f_stat, p_valor = stats.f_oneway(ingresos_CON_PA, ingresos_CON_PA_MECVV)

print("Estadística F:", f_stat)
print("Valor p:", p_valor)

# Interpretación del resultado
alpha = 0.05
if p_valor < alpha:
    print("Hay diferencias significativas entre los grupos.")
else:
    print("No hay diferencias significativas entre los grupos.")

Estadística F: 0.06922327005897648
Valor p: 0.7946273649012201
No hay diferencias significativas entre los grupos.


In [84]:
#Tukey
#TODO: es incorrecto?
CON_PA['grupo'] = 'CON_PA'
CON_PA_MECVV['grupo'] = 'CON_PA_MECVV'
SIN_PA['grupo'] = 'SIN_PA'

data = pd.concat([CON_PA, CON_PA_MECVV, SIN_PA], ignore_index=True)

# Realizar el test de Tukey solo si hay al menos dos grupos con datos
if len(data['grupo'].unique()) >= 2:
    # Realizar el test de Tukey
    tukey_results = pairwise_tukeyhsd(data['Num_ingresos'], data['grupo'], alpha=0.05)
    print(tukey_results)
else:
    print("Se requieren al menos dos grupos con datos para realizar comparaciones múltiples.")


C:\Users\maria\AppData\Local\Temp\ipykernel_23260\3680356222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CON_PA['grupo'] = 'CON_PA'
C:\Users\maria\AppData\Local\Temp\ipykernel_23260\3680356222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CON_PA_MECVV['grupo'] = 'CON_PA_MECVV'
C:\Users\maria\AppData\Local\Temp\ipykernel_23260\3680356222.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

      Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1       group2    meandiff p-adj   lower  upper  reject
---------------------------------------------------------------
      CON_PA CON_PA_MECVV   0.3118 0.9836 -3.9892 4.6127  False
      CON_PA       SIN_PA   1.1056 0.6603 -1.9321 4.1434  False
CON_PA_MECVV       SIN_PA   0.7939 0.8681  -2.951 4.5387  False
---------------------------------------------------------------


In [85]:
#T-test

In [86]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Num_ingresos'], CON_PA_MECVV['Num_ingresos'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.7689185970171732


In [87]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Num_ingresos'], SIN_PA['Num_ingresos'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.3199162306439434


In [88]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Num_ingresos'], SIN_PA['Num_ingresos'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.4255873112788504


## Disfàgia (ICD de disfàgia)

In [89]:
disf_CON_PA= (CON_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de disfagia: {disf_CON_PA:.2f}%")

Porcentaje de disfagia: 47.06%


In [90]:
disf_CON_PA_MECVV = (CON_PA_MECVV['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de disfagia: {disf_CON_PA_MECVV:.2f}%")

Porcentaje de disfagia: 30.00%


In [91]:
disf_SIN_PA= (SIN_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Porcentaje de disfagia: {disf_SIN_PA:.2f}%")

Porcentaje de disfagia: 18.37%


#### Pvalor

In [92]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['DO_diagnosticada'] == 1.0, CON_PA_MECVV['DO_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.3973619011700509


In [93]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['DO_diagnosticada'] == 1.0, SIN_PA['Num_ingresos'] == 1.0, equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.0016711485988959095


In [94]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['DO_diagnosticada'] == 1.0, SIN_PA['DO_diagnosticada'] == 1.0, equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_12)

P-valor entre CON_PA_MECVV y SIN_PA: 0.3973619011700509


In [95]:
#Comprovacion con Tukey
CON_PA['grupo'] = 'CON_PA'
CON_PA_MECVV['grupo'] = 'CON_PA_MECVV'
SIN_PA['grupo'] = 'SIN_PA'

data = pd.concat([CON_PA, CON_PA_MECVV, SIN_PA], ignore_index=True)

# Realizar el test de Tukey solo si hay al menos dos grupos con datos
if len(data['grupo'].unique()) >= 2:
    # Realizar el test de Tukey
    tukey_results = pairwise_tukeyhsd(data['DO_diagnosticada'] == 1.0, data['grupo'], alpha=0.05)
    print(tukey_results)
else:
    print("Se requieren al menos dos grupos con datos para realizar comparaciones múltiples.")

C:\Users\maria\AppData\Local\Temp\ipykernel_23260\1554346727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CON_PA['grupo'] = 'CON_PA'
C:\Users\maria\AppData\Local\Temp\ipykernel_23260\1554346727.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CON_PA_MECVV['grupo'] = 'CON_PA_MECVV'
C:\Users\maria\AppData\Local\Temp\ipykernel_23260\1554346727.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

      Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1       group2    meandiff p-adj   lower  upper  reject
---------------------------------------------------------------
      CON_PA CON_PA_MECVV  -0.1706 0.5863 -0.5834 0.2422  False
      CON_PA       SIN_PA  -0.2869 0.0547 -0.5785 0.0046  False
CON_PA_MECVV       SIN_PA  -0.1163 0.7199 -0.4757 0.2431  False
---------------------------------------------------------------


## Index de Barthel

### Promedio

In [96]:
promedio_barthel = CON_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = CON_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Promedio ± Desviación Estándar del Barthel:", resultado)

Promedio ± Desviación Estándar del Barthel: 22.65 ± 24.63


In [97]:
promedio_barthel = CON_PA_MECVV['Barthel_resultados'].mean()
desviacion_estandar_barthel = CON_PA_MECVV['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Promedio ± Desviación Estándar del Barthel:", resultado)

Promedio ± Desviación Estándar del Barthel: 28.50 ± 22.24


In [98]:
promedio_barthel = SIN_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = SIN_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Promedio ± Desviación Estándar del Barthel:", resultado)

Promedio ± Desviación Estándar del Barthel: 48.88 ± 37.42


#### Pvalor

In [99]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Barthel_resultados'], CON_PA_MECVV['Barthel_resultados'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: 0.5328822564529123


In [100]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Barthel_resultados'], SIN_PA['Barthel_resultados'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: 0.002116116594672647


In [101]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'], SIN_PA['Barthel_resultados'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: 0.03132528496286713


### Valores de dependencia

##### CON_PA

In [146]:
indepe_CON_PA = (CON_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Porcentaje de indepe_CON_PA: {indepe_CON_PA:.2f}%")

Porcentaje de indepe_CON_PA: 0.00%


In [251]:
mod_CON_PA = (CON_PA['Barthel_resultados'].between(61, 95)).mean() * 100
print(f"Porcentaje de mod_CON_PA: {mod_CON_PA:.2f}%")

Porcentaje de mod_CON_PA: 5.88%


In [149]:
sev_CON_PA = (CON_PA['Barthel_resultados'].between(21, 60)).mean() * 100
print(f"Porcentaje de sev_CON_PA: {sev_CON_PA:.2f}%")

Porcentaje de sev_CON_PA: 41.18%


In [157]:
total_CON_PA = (CON_PA['Barthel_resultados'] <= 20.0 ).mean() * 100
print(f"Porcentaje de total_CON_PA: {total_CON_PA:.2f}%")

Porcentaje de total_CON_PA: 52.94%


##### CON_PA_MECVV

In [168]:
indepe_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'] == 100.0).mean() * 100
print(f"Porcentaje de indepe_CON_PA_MECVV: {indepe_CON_PA_MECVV:.2f}%")

Porcentaje de indepe_CON_PA_MECVV: 0.00%


In [166]:
mod_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Porcentaje de mod_CON_PA_MECVV: {mod_CON_PA_MECVV:.2f}%")

Porcentaje de mod_CON_PA_MECVV: 0.00%


In [164]:
sev_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Porcentaje de sev_CON_PA_MECVV: {sev_CON_PA_MECVV:.2f}%")

Porcentaje de sev_CON_PA_MECVV: 60.00%


In [167]:
total_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'] <= 20.0).mean() * 100
print(f"Porcentaje de total_CON_PA_MECVV: {total_CON_PA_MECVV:.2f}%")

Porcentaje de total_CON_PA_MECVV: 40.00%


##### SIN_PA

In [169]:
indepe_SIN_PA = (SIN_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Porcentaje de indepe_SIN_PA: {indepe_SIN_PA:.2f}%")

Porcentaje de indepe_SIN_PA: 20.41%


In [170]:
mod_SIN_PA = (SIN_PA['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Porcentaje de mod_SIN_PA: {mod_SIN_PA:.2f}%")

Porcentaje de mod_SIN_PA: 18.37%


In [171]:
sev_SIN_PA = (SIN_PA['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Porcentaje de sev_SIN_PA: {sev_SIN_PA:.2f}%")

Porcentaje de sev_SIN_PA: 28.57%


In [173]:
total_SIN_PA = (SIN_PA['Barthel_resultados'] <= 20).mean() * 100
print(f"Porcentaje de total_SIN_PA: {total_SIN_PA:.2f}%")

Porcentaje de total_SIN_PA: 32.65%


#### Pvalor

##### Independencia

In [188]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Barthel_resultados'] == 100.0, CON_PA_MECVV['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_CON_PA y indepe_CON_PA_MECVV:", p_val_12)

P-valor entre indepe_CON_PA y indepe_CON_PA_MECVV: nan


In [189]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Barthel_resultados'] == 100.0, SIN_PA['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_CON_PA y indepe_SIN_PA:", p_val_13)

P-valor entre indepe_CON_PA y indepe_SIN_PA: 0.0009905991330133408


In [190]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'] == 100.0, SIN_PA['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre indepe_CON_PA_MECVV y SIN_PA: 0.0009905991330133408


##### Moderada

In [191]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Barthel_resultados'].between(61, 95), CON_PA_MECVV['Barthel_resultados'].between(61, 95), equal_var=False)
print("P-valor entre mod_CON_PA y mod_CON_PA_MECVV:", p_val_12)

P-valor entre mod_CON_PA y mod_CON_PA_MECVV: 0.3321949846529795


In [192]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Barthel_resultados'].between(61, 95), SIN_PA['Barthel_resultados'].between(61, 95), equal_var=False)
print("P-valor entre indepe_CON_PA y mod_SIN_PA:", p_val_13)

P-valor entre indepe_CON_PA y mod_SIN_PA: 0.1308021277917621


In [193]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'].between(61, 95), SIN_PA['Barthel_resultados'].between(61, 95), equal_var=False)
print("P-valor entre mod_CON_PA_MECVV y mod_SIN_PA :", p_val_23)

P-valor entre mod_CON_PA_MECVV y mod_SIN_PA : 0.0019018439578946838


##### Severa

In [194]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Barthel_resultados'].between(21,60), CON_PA_MECVV['Barthel_resultados'].between(21,60), equal_var=False)
print("P-valor entre sev_CON_PA y sev_CON_PA_MECVV:", p_val_12)

P-valor entre sev_CON_PA y sev_CON_PA_MECVV: 0.36895172812187915


In [195]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Barthel_resultados'].between(21,60), SIN_PA['Barthel_resultados'].between(21,60), equal_var=False)
print("P-valor entre sev_CON_PA y sev_SIN_PA:", p_val_13)

P-valor entre sev_CON_PA y sev_SIN_PA: 0.37379734622047833


In [196]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'].between(21,60), SIN_PA['Barthel_resultados'].between(21,60), equal_var=False)
print("P-valor entre sev_CON_PA_MECVV y sev_SIN_PA:", p_val_23)

P-valor entre sev_CON_PA_MECVV y sev_SIN_PA: 0.09905259977964401


##### Total

In [205]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Barthel_resultados'] <= 20.0, CON_PA_MECVV['Barthel_resultados'] <= 20.0, equal_var=False)
print("P-valor entre total_CON_PA y total_CON_PA_MECVV:", p_val_12)

P-valor entre total_CON_PA y total_CON_PA_MECVV: 0.536420664203854


In [209]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Barthel_resultados'] <= 20.0, SIN_PA['Barthel_resultados'] <= 20.0, equal_var=False)
print("P-valor entre total_CON_PA y total_SIN_PA:", p_val_13)

P-valor entre total_CON_PA y total_SIN_PA: 0.16484379406840946


In [208]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'] <= 20.0, SIN_PA['Barthel_resultados'] <= 20.0, equal_var=False)
print("P-valor entre total_CON_PA_MECVV y total_SIN_PA:", p_val_23)

P-valor entre total_CON_PA_MECVV y total_SIN_PA: 0.6848541420862003


## Index del MNA

Primero hay que convertir la columna a tipo numerico ya que es de tipo "object" y no permite realizar el promedio

In [213]:
CON_PA['mna_resultats'] = pd.to_numeric(CON_PA['mna_resultats'], errors='coerce')
CON_PA_MECVV['mna_resultats'] = pd.to_numeric(CON_PA_MECVV['mna_resultats'], errors='coerce')
SIN_PA['mna_resultats'] = pd.to_numeric(SIN_PA['mna_resultats'], errors='coerce')
#TODO: es incorrecto?

C:\Users\maria\AppData\Local\Temp\ipykernel_23260\4212325258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CON_PA['mna_resultats'] = pd.to_numeric(CON_PA['mna_resultats'], errors='coerce')
C:\Users\maria\AppData\Local\Temp\ipykernel_23260\4212325258.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CON_PA_MECVV['mna_resultats'] = pd.to_numeric(CON_PA_MECVV['mna_resultats'], errors='coerce')
C:\Users\maria\AppData\Local\Temp\ipykernel_23260\4212325258.py:3: SettingWithCopyWarning: 
A value is trying 

### Promedio

In [219]:
promedio_mna = CON_PA['mna_resultats'].mean()
desviacion_estandar_mna = CON_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Promedio ± Desviación Estándar del mna:", resultado)

Promedio ± Desviación Estándar del mna: 0.00 ± 0.00


In [221]:
promedio_mna = CON_PA_MECVV['mna_resultats'].mean()
desviacion_estandar_mna = CON_PA_MECVV['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Promedio ± Desviación Estándar del mna:", resultado)

Promedio ± Desviación Estándar del mna: 0.00 ± nan


In [216]:
promedio_mna = SIN_PA['mna_resultats'].mean()
desviacion_estandar_mna = SIN_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Promedio ± Desviación Estándar del mna:", resultado)

Promedio ± Desviación Estándar del mna: 3.05 ± 5.92


#### Pvalor

In [228]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['mna_resultats'], CON_PA_MECVV['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA y CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA y CON_PA_MECVV: nan


In [231]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['mna_resultats'], SIN_PA['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA y SIN_PA:", p_val_13)

P-valor entre CON_PA y SIN_PA: nan


In [232]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['mna_resultats'], SIN_PA['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA_MECVV y SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV y SIN_PA: nan


### Valores de malnutricion

##### CON_PA

In [252]:
norm_CON_PA = CON_PA['mna_resultats'].between(24, 30)
porcentaje_norm_CON_PA = norm_CON_PA.mean() * 100
num_observaciones_norm_CON_PA = norm_CON_PA.sum()

print(f"Porcentaje de norm_CON_PA: {porcentaje_norm_CON_PA:.2f}%")
print(f"Número de observaciones en norm_CON_PA: {num_observaciones_norm_CON_PA}")

Porcentaje de norm_CON_PA: 0.00%
Número de observaciones en norm_CON_PA: 0


In [253]:
malnutr_CON_PA = CON_PA['mna_resultats'].between(17, 23.5)
porcentaje_malnutr_CON_PA = malnutr_CON_PA.mean() * 100
num_observaciones_malnutr_CON_PA = norm_CON_PA.sum()

print(f"Porcentaje de malnutr_CON_PA: {porcentaje_malnutr_CON_PA:.2f}%")
print(f"Número de observaciones en norm_CON_PA: {num_observaciones_malnutr_CON_PA}")

Porcentaje de malnutr_CON_PA: 0.00%
Número de observaciones en norm_CON_PA: 0


In [258]:
malnut_CON_PA = CON_PA['mna_resultats'] < 17
porcentaje_malnut_CON_PA = malnut_CON_PA.mean() * 100
num_observaciones_malnut_CON_PA = malnut_CON_PA.sum()

print(f"Porcentaje de malnut_CON_PA: {porcentaje_malnut_CON_PA:.2f}%")
print(f"Número de observaciones en malnut_CON_PA: {num_observaciones_malnut_CON_PA}")

Porcentaje de malnut_CON_PA: 23.53%
Número de observaciones en malnut_CON_PA: 4


##### CON_PA_MECVV

In [259]:
norm_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'].between(24, 30)
porcentaje_norm_CON_PA_MECVV = norm_CON_PA_MECVV.mean() * 100
num_observaciones_norm_CON_PA_MECVV = norm_CON_PA_MECVV.sum()

print(f"Porcentaje de norm_CON_PA_MECVV: {porcentaje_norm_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en norm_CON_PA_MECVV: {num_observaciones_norm_CON_PA_MECVV}")

Porcentaje de norm_CON_PA_MECVV: 0.00%
Número de observaciones en norm_CON_PA_MECVV: 0


In [260]:
malnutr_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'].between(17, 23.5)
porcentaje_malnutr_CON_PA_MECVV = malnutr_CON_PA_MECVV.mean() * 100
num_observaciones_malnutr_CON_PA_MECVV = norm_CON_PA_MECVV.sum()

print(f"Porcentaje de malnutr_CON_PA_MECVV: {porcentaje_malnutr_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en norm_CON_PA_MECVV: {num_observaciones_malnutr_CON_PA_MECVV}")

Porcentaje de malnutr_CON_PA_MECVV: 0.00%
Número de observaciones en norm_CON_PA_MECVV: 0


In [261]:
malnut_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'] < 17
porcentaje_malnut_CON_PA_MECVV = malnut_CON_PA_MECVV.mean() * 100
num_observaciones_malnut_CON_PA_MECVV = malnut_CON_PA_MECVV.sum()

print(f"Porcentaje de malnut_CON_PA_MECVV: {porcentaje_malnut_CON_PA_MECVV:.2f}%")
print(f"Número de observaciones en malnut_CON_PA_MECVV: {num_observaciones_malnut_CON_PA_MECVV}")

Porcentaje de malnut_CON_PA_MECVV: 10.00%
Número de observaciones en malnut_CON_PA_MECVV: 1


##### SIN_PA

In [262]:
norm_SIN_PA = SIN_PA['mna_resultats'].between(24, 30)
porcentaje_norm_SIN_PA = norm_SIN_PA.mean() * 100
num_observaciones_norm_SIN_PA = norm_SIN_PA.sum()

print(f"Porcentaje de norm_SIN_PA: {porcentaje_norm_SIN_PA:.2f}%")
print(f"Número de observaciones en norm_SIN_PA: {num_observaciones_norm_SIN_PA}")

Porcentaje de norm_SIN_PA: 0.00%
Número de observaciones en norm_SIN_PA: 0


In [267]:
malnutr_SIN_PA = SIN_PA['mna_resultats'].between(17, 23.5)
porcentaje_malnutr_SIN_PA = malnutr_SIN_PA.mean() * 100
num_observaciones_malnutr_SIN_PA = malnutr_SIN_PA.sum()

print(f"Porcentaje de malnutr_SIN_PA: {porcentaje_malnutr_SIN_PA:.2f}%")
print(f"Número de observaciones en malnutr_SIN_PA: {num_observaciones_malnutr_SIN_PA}")

Porcentaje de malnutr_SIN_PA: 2.04%
Número de observaciones en malnutr_SIN_PA: 1


In [265]:
malnut_SIN_PA = SIN_PA['mna_resultats'] < 17
porcentaje_malnut_SIN_PA = malnut_SIN_PA.mean() * 100
num_observaciones_malnut_SIN_PA = malnut_SIN_PA.sum()

print(f"Porcentaje de malnut_SIN_PA: {porcentaje_malnut_SIN_PA:.2f}%")
print(f"Número de observaciones en malnut_SIN_PA: {num_observaciones_malnut_SIN_PA}")

Porcentaje de malnut_SIN_PA: 18.37%
Número de observaciones en malnut_SIN_PA: 9


#### Pvalor

##### Normal

In [268]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['mna_resultats'].between(24, 30), CON_PA_MECVV['mna_resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_CON_PA y norm_CON_PA_MECVV:", p_val_12)

P-valor entre norm_CON_PA y norm_CON_PA_MECVV: nan


In [269]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['mna_resultats'].between(24, 30), SIN_PA['mna_resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_CON_PA y norm_SIN_PA:", p_val_13)

P-valor entre norm_CON_PA y norm_SIN_PA: nan


In [270]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['mna_resultats'].between(24, 30), SIN_PA['mna_resultats'].between(24, 30), equal_var=False)
print("P-valor entre norm_CON_PA_MECVV y norm_SIN_PA:", p_val_23)

P-valor entre norm_CON_PA_MECVV y norm_SIN_PA: nan


##### Riesgo de malnutricion

In [276]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['mna_resultats'].between(17, 23.5), CON_PA_MECVV['mna_resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_CON_PA y malnutr_CON_PA_MECVV:", p_val_12)

P-valor entre malnutr_CON_PA y malnutr_CON_PA_MECVV: nan


In [277]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['mna_resultats'].between(17, 23.5), SIN_PA['mna_resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_CON_PA y malnutr_SIN_PA:", p_val_13)

P-valor entre malnutr_CON_PA y malnutr_SIN_PA: 0.3223252185211165


In [278]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['mna_resultats'].between(17, 23.5), SIN_PA['mna_resultats'].between(17, 23.5), equal_var=False)
print("P-valor entre malnutr_CON_PA_MECVV y malnutr_SIN_PA:", p_val_23)

P-valor entre malnutr_CON_PA_MECVV y malnutr_SIN_PA: 0.3223252185211165


##### Malnutricion

In [279]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['mna_resultats'] <17 , 
                                CON_PA_MECVV['mna_resultats'] < 17, equal_var=False)
print("P-valor entre malnut_CON_PA y norm_CON_PA_MECVV:", p_val_12)

P-valor entre malnut_CON_PA y norm_CON_PA_MECVV: 0.3626443145321582


In [280]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['mna_resultats'] <17 , 
                                SIN_PA['mna_resultats'] < 17, equal_var=False)
print("P-valor entre malnut_CON_PA y malnut_SIN_PA:", p_val_13)

P-valor entre malnut_CON_PA y malnut_SIN_PA: 0.6703598116552147


In [281]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['mna_resultats'] <17 , 
                                SIN_PA['mna_resultats'] < 17, equal_var=False)
print("P-valor entre malnut_CON_PA_MECVV y malnut_SIN_PA:", p_val_23)

P-valor entre malnut_CON_PA_MECVV y malnut_SIN_PA: 0.47623865054662595
